# A simple example of  multilable learning

In [1]:
%matplotlib inline

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd
import sklearn as sk
import cython
import itertools

from scipy.io import arff
from scipy.optimize import minimize
from scipy.optimize import check_grad

from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_dir = 'data'
yeast_ftrain = os.path.join(data_dir, 'yeast/yeast-train.arff')
yeast_ftest  = os.path.join(data_dir, 'yeast/yeast-test.arff')

## Data loading

Load yeast dataset.

In [3]:
data_train, meta_train = arff.loadarff(yeast_ftrain)

In [4]:
data_test, meta_test = arff.loadarff(yeast_ftest)

In [5]:
type(data_train)

numpy.ndarray

In [6]:
#print(data_train[0])

Features

In [7]:
nFeatures = np.array(list(data_train[0])[:-14], dtype=np.float).shape[0]
print('#features:', nFeatures)

#features: 103


Labels

In [8]:
nLabels = np.array(list(data_train[0])[-14:], dtype=np.int).shape[0]
print('#labels:', nLabels)

#labels: 14


## Data analysis

In [9]:
print('#training examples:', len(data_train))

#training examples: 1500


In [10]:
print('#test examples:', len(data_test))

#test examples: 917


## Dataset creation

In [11]:
def create_dataset(label_ix, data):
    """
        Create the labelled dataset for a given label index
        
        Input:
            - label_ix: label index, number in { 0, ..., # labels }
            - data: original data with features + labels
            
        Output:
            - (Feature, Label) pair (X, y)
              X comprises the features for each example
              y comprises the labels of the corresponding example
    """

    assert(label_ix >= 0)
    assert(label_ix < nLabels)

    N = len(data)
    d = nFeatures

    X = np.zeros((N, d), dtype = np.float)
    y = np.zeros(N, dtype = np.int)
       
    for i in range(N):
        X[i, :] = list(data[i])[:-14]
        y[i]    = list(data[i])[-14:][label_ix]

    return X, y

In [12]:
def create_dataset_v2(data):
    """
        Create the labelled dataset for a given label index
        
        Input:
            - data: original data with features + labels
            
        Output:
            - (Feature, Label) pair (X, y)
              X comprises the features for each example
              Y comprises the labels of the corresponding example
    """

    N = len(data)
    D = nFeatures
    L = nLabels

    X = np.zeros((N, D), dtype = np.float)
    Y = np.zeros((N, L), dtype = np.int)
       
    for i in range(N):
        X[i, :] = list(data[i])[:-14]
        Y[i, :] = list(data[i])[-14:]

    return X, Y

## Evaluation

Loss between a ground truth and a prediction.

In [13]:
def evalPred(truth, pred, lossType = 'Hamming'):
    """
        Compute loss given ground truth and prediction
        
        Input:
            - truth:    binary array of true labels
            - pred:     real-valued array of predictions
            - lossType: can be subset 0-1, Hamming, ranking, and Precision@K where K = # positive labels.
    """

    assert(len(truth) == len(pred))
    L = len(truth)
    nPos = np.sum(truth)
    
    predBin = np.array((pred > 0), dtype=np.int)
    
    if lossType == 'Subset01':
        return 1 - int(np.all(truth == predBin))
    
    elif lossType == 'Hamming':
        return np.sum(truth != predBin) / L
    
    elif lossType == 'Ranking':
        loss = 0
        for i in range(L-1):
            for j in range(i+1, L):
                if truth[i] > truth[j]:
                    if pred[i] < pred[j]: 
                        loss += 1
                    if pred[i] == pred[j]:
                        loss += 0.5
        #return loss / (nPos * (L-nPos))
        return loss
        
    elif lossType == 'Precision@K':
        # sorted indices of the labels most likely to be +'ve
        idx = np.argsort(pred)[::-1]
        
        # true labels according to the sorted order
        y = truth[idx]
        
        # fraction of +'ves in the top K predictions
        return np.mean(y[:nPos])
    
    elif lossType == 'Precision@3':
        # sorted indices of the labels most likely to be +'ve
        idx = np.argsort(pred)[::-1]
        
        # true labels according to the sorted order
        y = truth[idx]
        
        # fraction of +'ves in the top K predictions
        return np.mean(y[:3])
    
    elif lossType == 'Precision@5':
        # sorted indices of the labels most likely to be +'ve
        idx = np.argsort(pred)[::-1]
        
        # true labels according to the sorted order
        y = truth[idx]
        
        # fraction of +'ves in the top K predictions
        return np.mean(y[:5])
    
    else:
        assert(False)

In [14]:
def printEvaluation(allPreds, allTruths):

    for lossType in ['Subset01', 'Hamming', 'Ranking', 'Precision@K', 'Precision@3', 'Precision@5']:
        losses = [ ]
        for i in range(allPreds.shape[0]):
            pred  = allPreds[i, :]
            truth = allTruths[i, :]
            losses.append(evalPred(truth, pred, lossType))

            #print(allPreds[i])
            #print(pred)
            #print(truth)
            #break

        print('%24s: %1.4f' % ('Average %s Loss' % lossType, np.mean(losses)))
        #plt.hist(aucs, bins = 10);

## Binary relevance baseline

Train a logistic regression model for each label.

In [15]:
classifiers = [ LogisticRegression(class_weight = 'balanced', C = 10**0) for i in range(nLabels) ]

In [16]:
allPreds  = [ ]
allTruths = [ ]
coefMat = [ ]
labelIndices = [ ]

for label_ix in range(nLabels):
    X_train, y_train = create_dataset(label_ix, data = data_train)
    X_test, y_test   = create_dataset(label_ix, data = data_test)
    
    allTruths.append(y_test) 
    
    assert( (not np.all(y_train == 0)) or (not np.all(y_train == 1)) )

    classifiers[label_ix].fit(X_train, y_train)
    allPreds.append(classifiers[label_ix].decision_function(X_test))

    coefMat.append(classifiers[label_ix].coef_.reshape(-1))
    #labelIndices.append(label_ix)
    #print(classifiers[label_ix].coef_)
    #print(classifiers[label_ix].intercept_)

In [17]:
allPreds  = np.array(allPreds).T
allTruths = np.array(allTruths).T

print(allPreds.shape)
print(allTruths.shape)

(917, 14)
(917, 14)


In [18]:
#allPreds[0]

In [19]:
printEvaluation(allPreds, allTruths)

   Average Subset01 Loss: 0.9302
    Average Hamming Loss: 0.3325
    Average Ranking Loss: 5.2203
Average Precision@K Loss: 0.5149
Average Precision@3 Loss: 0.5209
Average Precision@5 Loss: 0.4733


### Result analysis

Coefficient matrix `(#Genres, #Songs)`.

In [20]:
coefMat = np.array(coefMat).T

In [21]:
coefMat.shape

(103, 14)

In [22]:
#sns.heatmap(coefMat[:, :30])

## Binary relevance with bipartite ranking

In [23]:
#%load_ext Cython

In [24]:
#%%cython -a

import numpy as np
#cimport numpy as np

#cpdef obj_biranking(w, X, y):

def obj_biranking(w, X, y):
    """
        Objective with L2 regularisation and bipartite ranking loss
        
        Input:
            - w: current weight vector
            - X: feature matrix, N x D
            - y: label vector,   N x 1
    """
    assert(len(y) == X.shape[0])
    assert(len(w) == X.shape[1])

    #cdef int nPos, nNeg, i, j
    #cdef double J, term, denom
    nPos = np.sum(y)      # num of positive examples
    nNeg = len(y) - nPos  # num of negative examples
    
    ixPos = np.nonzero(y)[0].tolist()                    # indices positive examples
    ixNeg = list(set(np.arange(len(y))) - set(ixPos))    # indices negative examples
    
    J = 0.0  # cost
    g = np.zeros_like(w)  # gradient    

    scorePos = X[ixPos, :].dot(w)[:,np.newaxis] # nPos x 1
    scoreNeg = X[ixNeg, :].dot(w)[:,np.newaxis] # nNeg x 1
    scoreDif = scorePos - scoreNeg.T            # nPos x nNeg
    #J = np.mean(np.log(1 + np.exp(-scoreDif)))
    J = 0.5 * np.dot(w, w) + np.mean(np.log1p(np.exp(-scoreDif)))
    
    A = -1/(1 + np.exp(scoreDif))

    T1 = X[ixPos, :].T.dot(A.sum(axis = 1))
    T2 = X[ixNeg, :].T.dot(A.sum(axis = 0))
    g  = w + 1/(nPos * nNeg) * (T1 - T2)
    
    return (J, g)

Check gradient.

In [25]:
X_train_, y_train_ = create_dataset(6, data = data_train)

In [26]:
w0 = w = np.random.rand(X_train_.shape[1])
check_grad(lambda w: obj_biranking(w, X_train_, y_train_)[0], \
           lambda w: obj_biranking(w, X_train_, y_train_)[1], w0)

1.0552253909154409e-06

In [27]:
1.1331503772158218e-06 * np.sqrt(nLabels)

4.2398604792352576e-06

In [28]:
params    = [ ]
allPreds  = [ ]
allTruths = [ ]

for label_ix in range(nLabels):
    #sys.stdout.write('\r%d / %d' % (label_ix + 1, nLabels))
    #sys.stdout.flush()
    print('\r%d / %d ' % (label_ix + 1, nLabels))
    
    X_train, y_train = create_dataset(label_ix, data = data_train)
    X_test, y_test   = create_dataset(label_ix, data = data_test)
    
    allTruths.append(y_test) 
    
    assert( (not np.all(y_train == 0)) or (not np.all(y_train == 1)) )
        
    opt_method = 'BFGS' #'Newton-CG' 
    #opt_method = 'nelder-mead'
    options = {'disp': True}
    
    w = np.random.rand(X_train.shape[1])  # initial guess
    opt = minimize(obj_biranking, w, args=(X_train, y_train), method=opt_method, jac=True, options=options)
    
    if opt.success == True:
        w = opt.x
        params.append(w)
        allPreds.append(np.dot(X_test, w))
    else:
        sys.stderr.write('Optimisation failed, label_ix=%d\n' % label_ix)
        w = np.zeros(X_train.shape[1])
        params.append(w)
        allPreds.append(np.dot(X_test, w))

1 / 14 
Optimization terminated successfully.
         Current function value: 0.687650
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
2 / 14 
Optimization terminated successfully.
         Current function value: 0.690264
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
3 / 14 
Optimization terminated successfully.
         Current function value: 0.687208
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
4 / 14 
Optimization terminated successfully.
         Current function value: 0.684365
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
5 / 14 
Optimization terminated successfully.
         Current function value: 0.687649
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6
6 / 14 
Optimization terminated successfully.
         Current function value: 0.689686
         Iterations: 5
    

In [29]:
allPreds = np.array(allPreds).T
allTruths = np.array(allTruths).T

print(allPreds.shape)
print(allTruths.shape)

(917, 14)
(917, 14)


In [30]:
#allPreds[0]

In [31]:
printEvaluation(allPreds, allTruths)

   Average Subset01 Loss: 0.9335
    Average Hamming Loss: 0.4301
    Average Ranking Loss: 7.2246
Average Precision@K Loss: 0.4400
Average Precision@3 Loss: 0.4427
Average Precision@5 Loss: 0.4100


## Ranking loss

In [32]:
#%load_ext Cython

In [33]:
#%%cython -a

import numpy as np
#cimport numpy as np

#cpdef obj_ranking(w, X, y):

def obj_ranking_loop(w, X, Y, C):
    """
        Objective with L2 regularisation and ranking loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - C: regularisation constant
    """
    N, D = X.shape
    L = Y.shape[1]
    assert(w.shape[0] == L * D)
    
    W = w.reshape(L, D)  # reshape weight matrix    
    
    #cdef int nPos, nNeg, i, j
    #cdef double J, term, denom
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    for n in range(N):
        Jn = 0.0
        Gn = np.zeros_like(W)
        x = X[n, :]
        y = Y[n, :]
        nPos = np.sum(y)   # num of positive examples
        nNeg = L - nPos    # num of negative examples
        denom = nPos * nNeg
        
        ixPos = np.nonzero(y)[0].tolist()               # indices positive examples
        ixNeg = list(set(np.arange(L)) - set(ixPos))    # indices negative examples
        
        for i in ixPos:
            for j in ixNeg:
                wDiff = W[i, :] - W[j, :]
                sDiff = np.dot(wDiff, x)
                term = np.exp(sDiff)
                Jn += np.log1p(1.0 / term)
                Gn[i, :] = Gn[i, :] - x / (1 + term)        
        #for j in ixNeg:
        #    for i in ixPos:
        #        wDiff = W[i, :] - W[j, :]
        #        sDiff = np.dot(wDiff, x)
        #        term = np.exp(sDiff)
                Gn[j, :] = Gn[j, :] + x / (1 + term)
                
        J += Jn / denom
        G = G + Gn / denom
        
    J = 0.5 * C * np.dot(w, w) + J / N
    G = C * W + G / N
    
    return (J, G.ravel())

In [34]:
#np.tile([1,2,3], (3,1)) * np.array([0.1, 0.2, 0.3])[:, None]

In [35]:
def obj_ranking(w, X, Y, C):
    """
        Objective with L2 regularisation and ranking loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - C: regularisation constant
    """
    N, D = X.shape
    L = Y.shape[1]
    assert(w.shape[0] == L * D)
    
    W = w.reshape(L, D)  # reshape weight matrix    
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    for n in range(N):
        Jn = 0.0
        Gn = np.zeros_like(W)
        x = X[n, :]
        y = Y[n, :]
        nPos = np.sum(y)   # num of positive examples
        nNeg = L - nPos    # num of negative examples
        denom = nPos * nNeg
        
        ixPos = np.nonzero(y)[0].tolist()               # indices positive examples
        ixNeg = list(set(np.arange(L)) - set(ixPos))    # indices negative examples
        
        ixmat = np.array(list(itertools.product(ixPos, ixNeg)))  # shape: ixPos*ixNeg by 2
        dW = W[ixmat[:, 0], :] - W[ixmat[:, 1], :]
        sVec = np.dot(dW, x)
        Jn = np.sum(np.log1p(np.exp(-sVec)))
        
        coeffVec = np.divide(1, 1 + np.exp(sVec))
        coeffPos = pd.DataFrame(coeffVec)
        coeffPos['gid'] = ixmat[:, 0]
        coeffPos = coeffPos.groupby('gid', sort=False).sum()
        coeffNeg = pd.DataFrame(coeffVec)
        coeffNeg['gid'] = ixmat[:, 1]
        coeffNeg = coeffNeg.groupby('gid', sort=False).sum()
        
        #print(coeffPos)
        #print(coeffNeg)
        
        coeffs = np.ones(L)
        coeffs[ixPos] = -coeffPos.loc[ixPos].values.squeeze()
        coeffs[ixNeg] = coeffNeg.loc[ixNeg].values.squeeze()
        
        #print(coeffs)
        Gn = np.tile(x, (L, 1)) * coeffs[:, None]
                        
        J += Jn / denom
        G = G + Gn / denom
        
    J = 0.5 * C * np.dot(w, w) + J / N
    G = C * W + G / N
    
    return (J, G.ravel())

In [36]:
X_train, Y_train = create_dataset_v2(data = data_train)
X_test,  Y_test  = create_dataset_v2(data = data_test)

Check gradient

In [37]:
#%%script false
C = 1
w0 = np.random.rand(nFeatures * nLabels)
check_grad(lambda w: obj_ranking(w, X_train[:20], Y_train[:20], C)[0], \
           lambda w: obj_ranking(w, X_train[:20], Y_train[:20], C)[1], w0)

5.023048495675116e-05

In [38]:
allPreds  = None
allTruths = Y_test

opt_method = 'BFGS' #'Newton-CG' 
#opt_method = 'nelder-mead'
options = {'disp': True}

C = 1
w = np.random.rand(nFeatures * nLabels)  # initial guess
opt = minimize(obj_ranking, w, args=(X_train, Y_train, C), method=opt_method, jac=True, options=options)

if opt.success == True:
    w = opt.x
    allPreds = np.dot(X_test, w.reshape(nLabels, nFeatures).T)
else:
    sys.stderr.write('Optimisation failed')

Optimization terminated successfully.
         Current function value: 0.692868
         Iterations: 3
         Function evaluations: 5
         Gradient evaluations: 5


In [39]:
#allPreds = np.array(allPreds).T
#allTruths = np.array(allTruths).T

print(allPreds.shape)
print(allTruths.shape)

(917, 14)
(917, 14)


In [40]:
#allPreds[0]

In [41]:
printEvaluation(allPreds, allTruths)

   Average Subset01 Loss: 0.9433
    Average Hamming Loss: 0.4329
    Average Ranking Loss: 6.7644
Average Precision@K Loss: 0.4901
Average Precision@3 Loss: 0.5075
Average Precision@5 Loss: 0.4412


## P-norm push loss

In [42]:
def obj_pnorm_push0(w, X, Y, p, C):
    """
        Objective with L2 regularisation and p-norm push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - p: constant for p-norm push loss
            - C: regularisation constant
    """
    N, D = X.shape
    L = Y.shape[1]
    assert(w.shape[0] == L * D)
    assert(p >= 1)
    assert(C >= 0)
    
    W = w.reshape(L, D)  # reshape weight matrix
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    for n in range(N):
        Gn = np.zeros_like(W)
        x = X[n, :]
        y = Y[n, :]
        nPos = np.sum(y)   # num of positive examples
        nNeg = L - nPos    # num of negative examples
        
        ixPos = np.nonzero(y)[0].tolist()               # indices positive examples
        ixNeg = list(set(np.arange(L)) - set(ixPos))    # indices negative examples
        
        # positive labels
        Jp = 0.0
        for k in ixPos:
            wk = W[k, :]
            term = np.exp(p * np.dot(wk, x))
            Jp += term / p
            Gn[k, :] = term * x / nPos
        J += Jp / nPos
        
        # negative labels
        Jn = 0.0
        for k in ixNeg:
            wk = W[k, :]
            term = np.exp(-np.dot(wk, x))
            Jn += term
            Gn[k, :] = -term * x / nNeg
        J += Jn / nNeg
        
        G = G + Gn
        
    J = 0.5 * C * np.dot(w, w) + J / N
    G = C * W + G / N
    
    return (J, G.ravel())

In [43]:
def obj_pnorm_push(w, X, Y, p, C):
    """
        Objective with L2 regularisation and p-norm push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - p: constant for p-norm push loss
            - C: regularisation constant
    """
    N, D = X.shape
    L = Y.shape[1]
    assert(w.shape[0] == L * D)
    assert(p >= 1)
    assert(C >= 0)
    
    W = w.reshape(L, D)  # reshape weight matrix
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    
    for n in range(N):
        Gn = np.zeros_like(W)
        x = X[n, :]
        y = Y[n, :]
        nPos = np.sum(y)   # num of positive examples
        nNeg = L - nPos    # num of negative examples
        
        for k in range(L):
            wk = W[k, :]
            term = np.dot(wk, x)
            if y[k] == 1:
                term2 = np.exp(p * term) / nPos
                J += term2 / p
                Gn[k, :] = x * term2
            else:
                term2 = np.exp(-term) / nNeg
                J += term2
                Gn[k, :] = -x * term2
        G = G + Gn
        
    J = 0.5 * C * np.dot(w, w) + J / N
    G = C * W + G / N
    
    return (J, G.ravel())

In [44]:
X_train, Y_train = create_dataset_v2(data = data_train)
X_test,  Y_test  = create_dataset_v2(data = data_test)

Check gradient

In [45]:
p = 1
C = 1
w0 = np.random.rand(nFeatures * nLabels)
check_grad(lambda w: obj_pnorm_push(w, X_train, Y_train, p, C)[0], \
           lambda w: obj_pnorm_push(w, X_train, Y_train, p, C)[1], w0)

3.5949397484521654e-05

In [46]:
allPreds  = None
allTruths = Y_test

opt_method = 'BFGS' #'Newton-CG' 
#opt_method = 'nelder-mead'
options = {'disp': True}

w = np.random.rand(nFeatures * nLabels)  # initial guess
p = 1  # [1, 10]
C = 0  # [0, 1]
opt = minimize(obj_pnorm_push, w, args=(X_train, Y_train, p, C), method=opt_method, jac=True, options=options)

if opt.success == True:
    w = opt.x
    allPreds = np.dot(X_test, w.reshape(nLabels, nFeatures).T)
else:
    sys.stderr.write('Optimisation failed')

Optimization terminated successfully.
         Current function value: 1.752710
         Iterations: 334
         Function evaluations: 336
         Gradient evaluations: 336


In [47]:
#allPreds = np.array(allPreds).T
#allTruths = np.array(allTruths).T

print(allPreds.shape)
print(allTruths.shape)

(917, 14)
(917, 14)


In [48]:
#allPreds[0]

In [49]:
printEvaluation(allPreds, allTruths)

   Average Subset01 Loss: 1.0000
    Average Hamming Loss: 0.5779
    Average Ranking Loss: 13.3064
Average Precision@K Loss: 0.2290
Average Precision@3 Loss: 0.2119
Average Precision@5 Loss: 0.2310


### Results for different hyper-parameter configurations

In [51]:
allTruths = Y_test
allTruths_train = Y_train
opt_method = 'BFGS' #'Newton-CG' 

for p in [1, 10]:
    for C in [0, 1]:
        print('\n-------------------------------------')
        print('p in loss: {}, C for regularisation: {}'.format(p, C))
        allPreds  = None

        w = np.random.rand(nFeatures * nLabels)  # initial guess
        opt = minimize(obj_pnorm_push, w, args=(X_train, Y_train, p, C), method=opt_method, jac=True)

        if opt.success == True:
            w = opt.x
            allPreds = np.dot(X_test, w.reshape(nLabels, nFeatures).T)
            allPreds_train = np.dot(X_train, w.reshape(nLabels, nFeatures).T)
        else:
            sys.stderr.write('Optimisation failed')
        print('Evaluation on training set:')
        printEvaluation(allPreds_train, allTruths_train)
        print()
        print('Evaluation on test set:')
        printEvaluation(allPreds, allTruths)


-------------------------------------
p in loss: 1, C for regularisation: 0
Evaluation on training set:
   Average Subset01 Loss: 1.0000
    Average Hamming Loss: 0.6474
    Average Ranking Loss: 15.1733
Average Precision@K Loss: 0.1505
Average Precision@3 Loss: 0.1211
Average Precision@5 Loss: 0.1565

Evaluation on test set:
   Average Subset01 Loss: 1.0000
    Average Hamming Loss: 0.5783
    Average Ranking Loss: 13.3119
Average Precision@K Loss: 0.2290
Average Precision@3 Loss: 0.2116
Average Precision@5 Loss: 0.2308

-------------------------------------
p in loss: 1, C for regularisation: 1
Evaluation on training set:
   Average Subset01 Loss: 0.9980
    Average Hamming Loss: 0.5813
    Average Ranking Loss: 13.5993
Average Precision@K Loss: 0.2499
Average Precision@3 Loss: 0.2313
Average Precision@5 Loss: 0.2509

Evaluation on test set:
   Average Subset01 Loss: 1.0000
    Average Hamming Loss: 0.5675
    Average Ranking Loss: 13.0905
Average Precision@K Loss: 0.2735
Average Pr